In [6]:
import pandas as pd
import mysql.connector
import os

# ----------------------------
# MySQL Connection
# ----------------------------
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='789025',
    database='adventureworks'
)
cursor = conn.cursor()

# ----------------------------
# File List and Folder Path
# ----------------------------
folder_path = r"D:/Harnek/Capstone/SQL/Cleaned_AW"

csv_files = [
    ('AdventureWorks_Customers.csv', 'Customers'),
    ('AdventureWorks_Product_Categories.csv', 'Product_Categories'),
    ('AdventureWorks_Product_Subcategories.csv', 'Product_Subcategories'),
    ('AdventureWorks_Products.csv', 'Products'),
    ('AdventureWorks_Returns.csv', 'Returns'),
    ('AdventureWorks_Territories.csv', 'Territories'),
    ('AdventureWorks_Sales_2015.csv', 'Sales_2015'),
    ('AdventureWorks_Sales_2016.csv', 'Sales_2016'),
    ('AdventureWorks_Sales_2017.csv', 'Sales_2017'),
    ('Sales.csv', 'Sales')
]

# ----------------------------
# Helper: Map pandas dtype → SQL type
# ----------------------------
def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

# ----------------------------
# Main Import Loop
# ----------------------------
for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    print(f"\n📂 Processing file: {csv_file}")

    # Try reading with UTF-8, fallback to Latin1 if needed
    try:
        df = pd.read_csv(file_path, encoding='utf-8')
    except UnicodeDecodeError:
        print("⚠️  UTF-8 decoding failed, retrying with latin1 encoding...")
        df = pd.read_csv(file_path, encoding='latin1')

    # Clean column names
    df.columns = [col.strip().replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    # Replace NaN with None for SQL compatibility
    df = df.where(pd.notnull(df), None)

    # Debug info
    print(f"➡️ Rows: {len(df)}, Columns: {len(df.columns)}")

    # ----------------------------
    # Create Table (if not exists)
    # ----------------------------
    columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
    cursor.execute(create_table_query)
    print(f"✅ Table '{table_name}' verified/created.")

    # ----------------------------
    # Insert Rows
    # ----------------------------
    insert_query = f"""
        INSERT INTO `{table_name}` ({', '.join([f'`{col}`' for col in df.columns])})
        VALUES ({', '.join(['%s'] * len(df.columns))})
    """

    for _, row in df.iterrows():
        values = tuple(None if pd.isna(x) else x for x in row)
        try:
            cursor.execute(insert_query, values)
        except mysql.connector.Error as err:
            print(f"⚠️ Skipping row due to error: {err}")

    conn.commit()
    print(f"💾 Data inserted into '{table_name}' successfully.")

# ----------------------------
# Close Connection
# ----------------------------
cursor.close()
conn.close()
print("\n🎉 All files processed and uploaded successfully!")



📂 Processing file: AdventureWorks_Returns.csv
➡️ Rows: 1809, Columns: 4
✅ Table 'Returns' verified/created.
💾 Data inserted into 'Returns' successfully.

🎉 All files processed and uploaded successfully!
